<div align="center">

<img src="https://raw.githubusercontent.com/ductho-le/WaveDL/main/logos/wavedl_logo.png" alt="WaveDL Logo" width="450">

# Interactive Demo

**Experience WaveDL in action — no installation required**

[![GitHub](https://img.shields.io/badge/⭐_Star_on_GitHub-181717?style=for-the-badge&logo=github&logoColor=white)](https://github.com/ductho-le/WaveDL)
[![License](https://img.shields.io/badge/License-MIT-F97316?style=for-the-badge)](https://github.com/ductho-le/WaveDL/blob/main/LICENSE)
[![Python](https://img.shields.io/badge/Python-3.11+-3776AB?style=for-the-badge&logo=python&logoColor=white)](https://www.python.org/)

</div>

---

## 🎯 What You'll Do

1. **Setup** — Clone repo & install dependencies
2. **Explore** — Visualize Lamb wave dispersion data
3. **Infer** — Run pre-trained CNN for material characterization
4. **Train** — (Optional) Quick training demo on synthetic data

---

### 📊 Demo Task: Material Characterization

This demo uses a **pre-trained CNN** to predict elastic properties from Lamb wave dispersion curves:

- **$h$** (mm) — Plate thickness
- **$\sqrt{E/\rho}$** (km/s) — Stiffness-to-density ratio  
- **$\nu$** — Lateral strain ratio (Poisson's ratio)

---

## 1️⃣ Setup Environment

In [ ]:
# 🚀 Run Setup - Clone WaveDL and install dependencies

import os
import sys
from IPython.display import clear_output

# Clone repository
if not os.path.exists('WaveDL'):
    print("📦 Cloning WaveDL repository...")
    !git clone --depth 1 https://github.com/ductho-le/WaveDL.git

%cd WaveDL

# Install WaveDL package
print("📥 Installing WaveDL package...")
!pip install -q -e .

# Force Python to recognize the new package (Colab workaround)
if 'wavedl' not in sys.modules:
    import importlib
    import site
    importlib.invalidate_caches()
    site.main()

clear_output()
print("✅ Setup Complete! WaveDL is ready.")

import torch
from wavedl import __version__
print(f"📦 WaveDL v{__version__}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("💻 Running on CPU")

---

## 2️⃣ Explore the Data

Each 500×500 pixel image encodes the wave propagation characteristics of a plate.

In [ ]:
# 📊 Visualize Dispersion Curves

import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams.update({'font.size': 11, 'figure.facecolor': 'white'})

test_data = Path("examples/elastic_cnn_example/Test_data_100.mat")

with h5py.File(test_data, 'r') as f:
    input_key = [k for k in f.keys() if 'input' in k.lower()][0]
    output_key = [k for k in f.keys() if 'output' in k.lower()][0]
    
    n_samples = f[input_key].shape[-1]
    input_shape = f[input_key].shape[:-1]
    n_targets = f[output_key].shape[0]
    
    np.random.seed(42)
    indices = np.random.choice(n_samples, 4, replace=False)
    samples = [f[input_key][:, :, i] for i in indices]
    targets = [f[output_key][:, i] for i in indices]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Sample Lamb Wave Dispersion Curves', fontsize=16, fontweight='bold', y=1.02)

param_names = [r'$h$', r'$\sqrt{E/\rho}$', r'$\nu$']
param_units = ['mm', 'km/s', '']

for idx, (ax, sample, target) in enumerate(zip(axes.flat, samples, targets)):
    im = ax.imshow(sample.T, aspect='auto', cmap='viridis', origin='lower')
    ax.set_xlabel('Frequency Index')
    ax.set_ylabel('Wavenumber Index')
    label = ' | '.join([f"{p}={v:.3f}{u}" for p, v, u in zip(param_names, target, param_units)])
    ax.set_title(f'Sample {indices[idx]+1}\n{label}', fontsize=10)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

print(f"\n📦 Dataset: {n_samples} samples × {input_shape[0]}×{input_shape[1]} pixels")
print(f"🎯 Targets: {n_targets} parameters per sample")

---

## 3️⃣ Run Inference

Use the pre-trained CNN to predict material properties and generate diagnostic plots.

In [ ]:
# 🧠 Run Pre-trained Model & View Results

from IPython.display import Image, display
from pathlib import Path
import subprocess
import os

os.makedirs('demo_results', exist_ok=True)

print("🔄 Running inference on 100 samples...\n")

# Use subprocess to avoid shell escaping issues with LaTeX
result = subprocess.run([
    'python', '-m', 'wavedl.test',
    '--checkpoint', './examples/elastic_cnn_example/best_checkpoint',
    '--data_path', './examples/elastic_cnn_example/Test_data_100.mat',
    '--param_names', r'$h$ (mm)', r'$\sqrt{E/\rho}$ (km/s)', r'$\nu$',
    '--plot',
    '--save_predictions',
    '--output_dir', './demo_results'
], capture_output=False)

# Display ALL diagnostic plots
results_dir = Path("demo_results")
plots = sorted(results_dir.glob("*.png"))

print("\n" + "="*60)
print(f"✅ Generated {len(plots)} diagnostic plots")
print("="*60)

for plot_path in plots:
    print(f"\n📊 {plot_path.stem.replace('_', ' ').title()}")
    display(Image(filename=str(plot_path), width=700))

---

## 4️⃣ Quick Training Demo (Optional)

Try the training pipeline with synthetic data.

In [ ]:
# 🎲 Generate Synthetic Data & Train

import numpy as np
import os

# Create synthetic dataset
np.random.seed(2025)
N_SAMPLES, IMG_SIZE, N_TARGETS = 200, 64, 3

X = np.random.randn(N_SAMPLES, IMG_SIZE, IMG_SIZE).astype(np.float32)
y = np.random.randn(N_SAMPLES, N_TARGETS).astype(np.float32)
np.savez('demo_train.npz', input_train=X, output_train=y)

print(f"✅ Created: {N_SAMPLES} samples × {IMG_SIZE}×{IMG_SIZE} → {N_TARGETS} targets")
print("\n🏋️ Training for 5 epochs...\n")

!python -m wavedl.train \
    --model cnn \
    --data_path demo_train.npz \
    --batch_size 32 \
    --epochs 5 \
    --patience 3 \
    --lr 1e-3 \
    --output_dir demo_training

print("\n✅ Training complete!")

---

<br>

<div align="center">

<img src="https://raw.githubusercontent.com/ductho-le/WaveDL/main/logos/wavedl_logo.png" alt="WaveDL" width="150">

## 🎉 Demo Complete!

**You've experienced the full WaveDL workflow**

</div>

### 🚀 What's Next?

- 📖 **[Documentation](https://github.com/ductho-le/WaveDL)** — Full README & guides
- 🔧 **[Custom Models](https://github.com/ductho-le/WaveDL/blob/main/src/wavedl/models/_template.py)** — Add your architecture
- 📊 **[Data Format](https://github.com/ductho-le/WaveDL#-data-preparation)** — Prepare your dataset

### 💬 Get Help

- [Ask a Question](https://github.com/ductho-le/WaveDL/discussions) | [Report an Issue](https://github.com/ductho-le/WaveDL/issues)

---

<div align="center">

⭐ **Enjoying WaveDL?** [Star us on GitHub!](https://github.com/ductho-le/WaveDL)

*Made by [Ductho Le](https://github.com/ductho-le)*

</div>